In [1]:
import pandas as pd
#need to get access to current fire data from NASA FIRMS, following instructions from NASA FIRMS: https://firms.modaps.eosdis.nasa.gov/content/academy/data_api/firms_api_use.html
# Let's set your map key that was emailed to you. It should look something like 'abcdef1234567890abcdef1234567890'
MAP_KEY = '8a9f5ecbad092d21416c6105e7313d55'
def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

# now let's check how many transactions we have
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)

# let's query data_availability to find out what date range is available for various datasets
# we will explain these datasets a bit later

# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/VIIRS_SNPP_SP'
VIIRS_SNPP_SP = pd.read_csv(da_url)
display(VIIRS_SNPP_SP)

# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

Error in our call.
Our current transaction count is 0


transaction_limit             5000
current_transactions             0
transaction_interval    10 minutes
dtype: object

,data_id,min_date,max_date
0,VIIRS_SNPP_SP,2012-01-20,2024-07-31


We used 1 transactions.


In [21]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
import pandas as pd
df_sample = pd.read_csv('https://firms.modaps.eosdis.nasa.gov/content/notebooks/sample_viirs_snpp_071223.csv')

sample_map = folium.Map(location = middle_coords, zoom_start = 5)


# show top 5 records
df_sample.head()
print(df_sample.columns)

#filtering latitude & longitude to keep values around the US
df_filtered = df_sample[(df_sample['latitude'] >= 32) & (df_sample['latitude'] <= 49)]
df_filtered = df_filtered[(df_filtered['longitude'] >= -124) & (df_filtered['longitude'] <= -115)]

df_filtered.head()

middle_coords = [39.5, -98.35]
sample_map = folium.Map(location = middle_coords, zoom_start = 5)

for index, row in df_filtered.iterrows():
    # Define rectangle bounds (a small box around the latitude/longitude)
    bounds = [[row['latitude'] - 0.01, row['longitude'] - 0.01], [row['latitude'] + 0.01, row['longitude'] + 0.01]]
    folium.Rectangle(bounds=bounds, color="red",  fill=True, fill_color="red",fill_opacity=0.5, tooltip="Fire").add_to(sample_map)

# Add a marker with the popup containing the textbox
dataset_name = "Sample VIIRS Snpp - 07/12/23"
html_content = f'<b><span style="background-color: yellow; padding: 5px;">{dataset_name}</span></b>'

folium.map.Marker(
    [30, -120],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<b><span style="background-color: yellow; padding: 5px; font-size: 20pt;">{dataset_name}</span></b>',
        )
    ).add_to(sample_map)


display(sample_map)



Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'daynight'],
      dtype='object')


trying to get text to show up on folium map

### Trying to get USA data for california

In [36]:
# Let's see last four days VIIRS data for USA
import pandas as pd
import folium 

USA_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/8a9f5ecbad092d21416c6105e7313d55/VIIRS_SNPP_NRT/-125,24,-66,49/1/2025-01-10'
df_USA = pd.read_csv(USA_url)
print(USA_url)
df_USA
# Split the URL and extract the parts
parts = USA_url.split('/')

# Extract the necessary parts
sensor = parts[7]  # VIIRS_SNPP_NRT
date = parts[10]   # 2025-01-10
dataset_name = str(sensor) + " - " + str(date) #"VIIRS SNPP NRT - 01/10/25"


middle_coords = [39.5, -98.35]
MVP1map = folium.Map(location = middle_coords, zoom_start = 4)

for index, row in df_USA.iterrows():
    # Define rectangle bounds (a small box around the latitude/longitude)
    bounds = [[row['latitude'] - 0.01, row['longitude'] - 0.01], [row['latitude'] + 0.01, row['longitude'] + 0.01]]
    folium.Rectangle(bounds=bounds, color="red",  fill=True, fill_color="red",fill_opacity=0.5, tooltip="Fire").add_to(MVP1map)

# Add a marker with the popup containing the textbox
folium.map.Marker(
    [30, -120],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<b><span style="background-color: orange; padding: 5px; font-size: 16pt;">{dataset_name}</span></b>',
        )
    ).add_to(MVP1map)



display(MVP1map)



https://firms.modaps.eosdis.nasa.gov/api/area/csv/8a9f5ecbad092d21416c6105e7313d55/VIIRS_SNPP_NRT/-125,24,-66,49/1/2025-01-10


#### Trying a different way of adding the HTML

In [37]:
import folium

# Create the base map
MVP1map = folium.Map(location=[39.5, -98.35], zoom_start=5)

# Define the fixed text as an HTML block
custom_html = """
<div style="
    position: fixed; 
    bottom: 10px; left: 10px; 
    width: 250px; 
    height: auto; 
    background-color: orange; 
    padding: 10px; 
    border: 2px solid black;
    font-size: 16pt;
    z-index: 1000;">
    Dataset Name: <b>VIIRS_SNPP_NRT</b>
</div>
"""

# Save the map to an HTML file
MVP1map.save('map_with_text.html')

# Open the generated HTML file and append the custom HTML
with open('map_with_text.html', 'r') as file:
    map_html = file.read()

# Insert the custom HTML before the closing </body> tag
map_html = map_html.replace('</body>', f'{custom_html}</body>')

# Save the modified HTML back to the file
with open('map_with_text.html', 'w') as file:
    file.write(map_html)

print("Map with fixed text saved as 'map_with_text.html'")


Map with fixed text saved as 'map_with_text.html'
